# 1. Knihovna `threading`

Modul `threading` umožňuje spouštět více vláken v rámci jednoho procesu. Vlákna sdílejí paměť, takže mohou jednoduše pracovat nad stejnými daty.

V CPythonu ale platí GIL: v jeden okamžik běží Python byte-kód pouze v jednom vlákně. Proto vlákna obvykle nepřinesou zrychlení pro CPU-bound Python kód. Naopak dávají smysl pro I/O-bound úlohy nebo pro kód, který běží mimo GIL.

Další užitečné nástroje v modulu:
- `threading.Condition`
- `threading.Semaphore`
- `threading.Timer`
- `threading.Event`

## 1.1 Vytvoření a spuštění vlákna

Vlákno vytvoříme přes `threading.Thread`. Důležitý parametr je `target`, tedy funkce, kterou má vlákno spustit.

Často používané parametry:
- `args`/`kwargs` pro argumenty cílové funkce,
- `name` pro pojmenování vlákna.

Základní metody:
- `start()` spustí vlákno,
- `join()` čeká na jeho dokončení,
- `is_alive()` vrací `True`, pokud vlákno stále běží.

In [ ]:
import threading


def worker():
    print("Hello from thread!")


thread = threading.Thread(target=worker)
thread.start()
thread.join()

Ukázka předání argumentů do vlákna:

In [ ]:
import threading
import time


def vypocet_soucet(vystupy, index):
    soucet = 0
    for i in range(20_000_000):
        soucet += i
    vystupy[index] = soucet


start = time.time()

vystupy = [0]
vlakno = threading.Thread(target=vypocet_soucet, args=(vystupy, 0))

vlakno.start()
vlakno.join()

konec = time.time()
print("Doba trvání:", konec - start)

In [ ]:
import threading
import time


def vypocet_soucet(vystupy, index):
    soucet = 0
    for i in range(20_000_000):
        soucet += i
    vystupy[index] = soucet


start = time.time()

vystupy = [0, 0]
vlakno1 = threading.Thread(target=vypocet_soucet, args=(vystupy, 0))
vlakno2 = threading.Thread(target=vypocet_soucet, kwargs={"vystupy": vystupy, "index": 1})

vlakno1.start()
vlakno2.start()

vlakno1.join()
vlakno2.join()

konec = time.time()
print("Doba trvání:", konec - start)

V ukázce výše se obě vlákna u čistě Python výpočtu střídají na GIL, takže celkový čas bývá blízko součtu časů jednotlivých běhů (plus režie).

## 1.2 Vlákna a kód mimo GIL

Teď stejný princip vyzkoušíme s funkcí přeloženou Numbou s `nogil=True`.

In [ ]:
from numba import jit

@jit(nopython=True, nogil=True)
def core():
    a = 0
    n = 3_000
    for i in range(n):
        for j in range(n):
            a += 2 * i * j
            a -= i * i
            a -= j * j
    return a

In [ ]:
%time res = core()

In [ ]:
res

In [ ]:
def vypocet_numba(vystupy, index):
    vystupy[index] = core()


start = time.time()

vystupy = [0, 0]
vlakno1 = threading.Thread(target=vypocet_numba, args=(vystupy, 0))
vlakno2 = threading.Thread(target=vypocet_numba, args=(vystupy, 1))

vlakno1.start()
vlakno2.start()

vlakno1.join()
vlakno2.join()

konec = time.time()
print("Doba trvání:", konec - start)
vystupy

## 1.3 Vlákna a sdílená data

In [ ]:
import random
import threading
import time

data = []  # sdílená datová struktura


def append_to_data(item):
    time.sleep(random.randint(0, 10) / 1000)
    data.append(item)


items = [i for i in range(20)]

threads = []
for item in items:
    thread = threading.Thread(target=append_to_data, args=(item,))
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()

print(f"Sdílená data: {data}")

## 1.4 Kritická sekce a `Lock`

`Lock` je vzájemné vyloučení (mutual exclusion): v jednom okamžiku může kritickou sekci vykonávat právě jedno vlákno.

In [ ]:
import threading

lock = threading.Lock()
shared_counter = 0


def increment_shared_counter():
    global shared_counter
    for _ in range(100_000):
        with lock:
            shared_counter += 1


threads = []
for _ in range(10):
    thread = threading.Thread(target=increment_shared_counter)
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()

print(f"Hodnota sdíleného čítače: {shared_counter}")

In [ ]:
import threading
import time


def vypis_bez_locku(thread_id):
    for i in range(5):
        time.sleep(0.001)
        print(f"Vlákno {thread_id} vypisuje {i}")


threads = []
for idx in range(3):
    thread = threading.Thread(target=vypis_bez_locku, args=(idx,))
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()

In [ ]:
import threading
import time

lock = threading.Lock()


def vypis_s_lockem(thread_id):
    with lock:
        for i in range(5):
            time.sleep(0.001)
            print(f"Vlákno {thread_id} vypisuje {i}")


threads = []
for idx in range(3):
    thread = threading.Thread(target=vypis_s_lockem, args=(idx,))
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()

## 1.5 Synchronizace vláken přes `Barrier`

`Barrier` umožní skupině vláken čekat na společný bod. Teprve když dorazí všechna vlákna, pokračují dál.

In [ ]:
import threading
import time


def worker(barrier, worker_id):
    print(f"Vlákno {worker_id} začíná.")
    time.sleep(worker_id * 0.3)
    print(f"Vlákno {worker_id} dosáhlo bariéry č. 1.")
    barrier.wait()
    print(f"Vlákno {worker_id} opouští bariéru č. 1.")

    time.sleep(worker_id * 0.3)
    print(f"Vlákno {worker_id} dosáhlo bariéry č. 2.")
    barrier.wait()
    print(f"Vlákno {worker_id} opouští bariéru č. 2.")


number_of_threads = 4
barrier = threading.Barrier(number_of_threads)

threads = []
for i in range(number_of_threads):
    thread = threading.Thread(target=worker, args=(barrier, i))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()